In [1]:
pip install mediapipe opencv-python speechrecognition pyaudio python-osc numpy pygame

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/b7/79/b77808f8195f229ef0c15875540dfdd36724748a4b3de53d993f23336839/mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for speechrecognition from https://files.pythonhosted.org/packages/aa/cd/4b5f5d04c8a4e25c376858d0ad28c325f079f17c82bf379185abf45e41bf/speechrecognition-3.14.3-py3-none-any.whl.metadata
  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/b0/6a/d25812e5f79f06285767ec607b39149d02aa3b31d50c2269768f48768930/PyAudio-0.2.14-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for python-osc from https://files.pythonhosted.org/packages/57/04/01b3de6a7565cfc20f3bf4fce959b099ad

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import mediapipe as mp
import speech_recognition as sr
import threading
import pygame
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Estado compartido
estado_gesto = {"mano_abierta": False, "dos_dedos": False}
ultimo_comando = ""
x = 100
color = (255, 255, 255)
mostrar = True

# Función para detectar gestos
def detectar_gestos():
    global estado_gesto
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        estado_gesto = {"mano_abierta": False, "dos_dedos": False}
        # Detección simple basada en landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                dedos_arriba = 0
                if hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y:
                    dedos_arriba += 1  # Índice
                if hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y:
                    dedos_arriba += 1  # Medio
                if hand_landmarks.landmark[16].y < hand_landmarks.landmark[14].y:
                    dedos_arriba += 1  # Anular
                if hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y:
                    dedos_arriba += 1  # Meñique
                if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
                    dedos_arriba += 1  # Pulgar (posición relativa)

                estado_gesto["mano_abierta"] = dedos_arriba >= 4
                estado_gesto["dos_dedos"] = dedos_arriba == 2

        cv2.imshow("Camara - Gesto", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Función para reconocimiento de voz
def reconocer_comando():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        try:
            print("Escuchando comando...")
            audio = r.listen(source, timeout=3)
            texto = r.recognize_google(audio, language="es-ES")
            print("Comando reconocido:", texto)
            return texto.lower()
        except:
            return ""

def escuchar_voz():
    global ultimo_comando
    while True:
        comando = reconocer_comando()
        if comando:
            ultimo_comando = comando

pygame.init()
pantalla = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Interfaz Multimodal")
font = pygame.font.SysFont("Arial", 30)
reloj = pygame.time.Clock()

threading.Thread(target=detectar_gestos, daemon=True).start()
threading.Thread(target=escuchar_voz, daemon=True).start()


while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()

    if estado_gesto["mano_abierta"]:
        if "cambiar" in ultimo_comando:
            color = (0, 0, 255)
        elif "rojo" in ultimo_comando:
            color = (255, 0, 0)
        elif "verde" in ultimo_comando:
            color = (0, 255, 0)

    if estado_gesto["dos_dedos"]:
        if "mover" in ultimo_comando:
            x = (x + 5) % 800
        elif "ocultar" in ultimo_comando:
            mostrar = False
        elif "mostrar" in ultimo_comando:
            mostrar = True

    # Renderizar escena
    pantalla.fill((0, 0, 0))
    if mostrar:
        pygame.draw.circle(pantalla, color, (x, 300), 50)
    texto = font.render(f"Comando: {ultimo_comando}", True, (255, 255, 255))
    pantalla.blit(texto, (10, 10))

    pygame.display.flip()
    reloj.tick(30)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Comando reconocido: It's como
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Comando reconocido: cambiar
Escuchando comando...
Escuchando comando...
Escuchando comando...
Comando reconocido: rojo
Escuchando comando...
Comando reconocido: amarillo
Escuchando comando...
Escuchando comando...
Comando reconocido: verde
Escuchando comando...
Escuchando comando...
Comando reconocido: amarillo
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Comando reconocido: rojo
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando comando...
Escuchando co

error: display Surface quit

: 